In [19]:
import pandas as pd
import os

# WNBA Playoff Prediction

## Data Understanding

Importing datasets

In [20]:
dataframes = dict()
for dataset_name in os.listdir('data'):
    if dataset_name.endswith('.csv'):
        dataframes[dataset_name.split(".")[0]] = pd.read_csv('data/' + dataset_name) 

## Data Preparation

### Select Data

Eliminating columns with exclusively unique values

In [21]:
for df_name, df in dataframes.items():
    df_clean = df.copy()
    for col in df.columns:
        if len(pd.unique(df[col])) == 1:
            df_clean.drop(col, axis=1, inplace=True)
    dataframes[df_name] = df_clean


### Integrate Data

Merging all dataframes into a single one

In [22]:
data = pd.merge(dataframes['teams_post'], dataframes['teams'], on=['tmID', 'year'], how='right')
data = pd.merge(data, dataframes['coaches'], on=['tmID', 'year'], suffixes=('_team', '_coach'))
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDWinner', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['series_post'], left_on=['tmID', 'year'], right_on=['tmIDLoser', 'year'], suffixes=('_team_post', '_series'), how='left')
data = pd.merge(data, dataframes['players_teams'], on=['tmID', 'year'], suffixes=('_team', '_player'))
data = pd.merge(data, dataframes['players'], left_on='playerID', right_on='bioID')
data = pd.merge(data, dataframes['awards_players'], on=['playerID'], how='left')

In [23]:
data.shape

(2943, 122)

In [24]:
label = "playoff"

In [25]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier
from sklearn.model_selection import train_test_split # Import train_test_split function
from sklearn import metrics #Import scikit-learn metrics module for accuracy calculation


In [26]:
df = dataframes["teams"]
df.head(3)

,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,GP,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena
0,9,ATL,ATL,EA,7,N,NaN,NaN,NaN,Atlanta Dream,...,34,1,16,3,14,2,18,6825,141379,Philips Arena
1,10,ATL,ATL,EA,2,Y,L,NaN,NaN,Atlanta Dream,...,34,12,5,6,11,10,12,6950,120737,Philips Arena
2,1,CHA,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,...,32,5,11,3,13,5,16,6475,90963,Charlotte Coliseum


In [31]:
features = df.columns
features = features.drop(label)
assert len(features) < len(df.columns)
X = df[df.columns]  # Features
X.head(3)

,year,tmID,franchID,confID,rank,playoff,firstRound,semis,finals,name,...,GP,homeW,homeL,awayW,awayL,confW,confL,min,attend,arena
0,9,ATL,ATL,EA,7,N,NaN,NaN,NaN,Atlanta Dream,...,34,1,16,3,14,2,18,6825,141379,Philips Arena
1,10,ATL,ATL,EA,2,Y,L,NaN,NaN,Atlanta Dream,...,34,12,5,6,11,10,12,6950,120737,Philips Arena
2,1,CHA,CHA,EA,8,N,NaN,NaN,NaN,Charlotte Sting,...,32,5,11,3,13,5,16,6475,90963,Charlotte Coliseum


In [30]:
y  =df[label]# Targe
y.head(3)

0    N
1    Y
2    N
Name: playoff, dtype: object